<a href="https://colab.research.google.com/github/advik-7/NLP_projects/blob/main/fine_tuning_distilberwidLORA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch transformers datasets scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import torch
import torch.nn as nn
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.metrics import accuracy_score, f1_score

# Check if GPU is available and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load AG News dataset
dataset = load_dataset("ag_news")

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenization function
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

# Apply tokenization and set format
dataset = dataset.map(tokenize, batched=True)
dataset = dataset.rename_column("label", "labels")
dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Define LoRA Layer for Low-Rank Adaptation
class LoRALayer(nn.Module):
    def __init__(self, in_features, out_features, rank=4):
        super(LoRALayer, self).__init__()
        self.down_proj = nn.Linear(in_features, rank, bias=False)
        self.up_proj = nn.Linear(rank, out_features, bias=False)

    def forward(self, x):
        return self.up_proj(self.down_proj(x))

# Modify DistilBERT to include LoRA layers in the attention heads
class LoRADistilBERT(nn.Module):
    def __init__(self, distilbert_model, rank=4):
        super(LoRADistilBERT, self).__init__()
        self.distilbert = distilbert_model

        # Replace attention layers with LoRA layers
        for name, module in self.distilbert.transformer.layer.named_modules():
            if name.endswith("attention.q_lin"):
                in_features, out_features = module.in_features, module.out_features
                lora_layer = LoRALayer(in_features, out_features, rank=rank)
                setattr(self.distilbert.transformer.layer, name, lora_layer)

    def forward(self, input_ids, attention_mask=None, labels=None):
        return self.distilbert(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

# Load pre-trained DistilBERT model and apply LoRA modifications
distilbert = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)
model = LoRADistilBERT(distilbert)
model.to(device)

# Metrics for evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": accuracy, "f1": f1}

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True
)

# Initialize Trainer with model and data
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"].select(range(500)),  # Limit to 500 samples for practice
    eval_dataset=dataset["test"].select(range(100)),   # Limit to 100 samples for practice
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print("Test Results:", results)


Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

...

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttributeError: 'DistilBertForSequenceClassification' object has no attribute 'transformer'

In [4]:
# Modify DistilBERT to include LoRA layers in the attention heads
class LoRADistilBERT(nn.Module):
    def __init__(self, distilbert_model, rank=4):
        super(LoRADistilBERT, self).__init__()
        self.distilbert = distilbert_model.distilbert  # Access base transformer layers
        self.classifier = distilbert_model.classifier

        # Replace attention layers with LoRA layers
        for layer in self.distilbert.transformer.layer:
            in_features, out_features = layer.attention.q_lin.in_features, layer.attention.q_lin.out_features
            layer.attention.q_lin = LoRALayer(in_features, out_features, rank=rank)

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.last_hidden_state[:, 0, :])  # Classifier on [CLS] token output
        return torch.nn.functional.cross_entropy(logits, labels) if labels is not None else logits


# Load pre-trained DistilBERT model and apply LoRA modifications
distilbert = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)
model = LoRADistilBERT(distilbert)
model.to(device)

# Metrics for evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": accuracy, "f1": f1}

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True
)

# Initialize Trainer with model and data
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"].select(range(500)),  # Limit to 500 samples for practice
    eval_dataset=dataset["test"].select(range(100)),   # Limit to 100 samples for practice
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print("Test Results:", results)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 3


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Abort: 

In [7]:
from torch.utils.data import DataLoader
from torch.optim import AdamW
import torch

# Define optimizer
optimizer = AdamW(model.parameters(), lr=1e-4)


    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


Epoch 1, Loss: 2.3117177486419678
Epoch 2, Loss: 4.293351650238037
Epoch 3, Loss: 1.6298283338546753


In [9]:

optimizer = AdamW(model.parameters(), lr=5e-6)


In [10]:

# Training loop
model.train()
for epoch in range(10):
    for batch in train_dataloader:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        # Check if the output is a tuple with loss as the first element or just the loss
        if isinstance(outputs, tuple):
            # If it's a tuple (loss, logits)
            loss = outputs[0]
        else:
            # If it's just the loss
            loss = outputs

        # Backward pass
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


Epoch 1, Loss: 3.5418100357055664
Epoch 2, Loss: 3.51240611076355
Epoch 3, Loss: 3.4717671871185303
Epoch 4, Loss: 3.4287099838256836
Epoch 5, Loss: 3.5165562629699707
Epoch 6, Loss: 3.4337897300720215
Epoch 7, Loss: 3.4048819541931152
Epoch 8, Loss: 3.380728244781494
Epoch 9, Loss: 3.377145767211914
Epoch 10, Loss: 3.3931360244750977
